## An Implementation of Ant Colony Optimization for the Traveling Salesperson Problem

## 1. Introduction

Ant Colony Optimization (ACO), which is introduced by Dorigo (1992; 1999), is a recent meta-heuristic approach for solving hard combinatorial optimization (CO) problems. The inspiring source of ACO is the pheromone trail laying and following behavior of real ants which use pheromones as a communication medium (Dorigo and Stutzle, 2003). This behaviour enables them to find short paths between their nest and food sources. This characteristic of real ant colonies is exploited in ACO algorithms in order to solve, for example, discrete optimization problems (Blum, 2005).

Seen from the operations research (OR) perspective, ACO algorithms belong to the class of meta-heuristics (Blum, 2005). Meta-heuristics for optimization problems may be described summarily as a "walk through neighbourhoods", a search trajectory through the solution domain of the problem at hand. Similar to classical heuristics, these are iterative procedures that move from a given solution to another solution in its neighbourhood (Crainic and Toulouse, 2003).

Artificial ants used in ACO are stochastic solution construction procedures that probabilistically build a solution by iteratively adding solution components to partial solutions by taking into account (i) heuristic information on the problem instance being solved, if available, and (ii) (artificial) pheromone trails which change dynamically at run-time to reflect the agents’ acquired search experience (Dorigo and Stutzle, 2003).

The first ACO algorithm proposed was Ant System (AS). AS was applied to some rather small instances of the traveling salesperson problem (TSP) (Dorigo and Di Caro, 1999). A brief introduction about AS algorithm and TSP is given with following sections, and finally AS is implemented for TSP.

## 2. Ant System

AS was the first example of an ACO algorithm to be proposed in the literature whose computational results were promising but not competitive with other more established approaches (Dorigo and Stutzle, 2003).

Each meta-heuristic has its own behaviour and characteristics. All, however, share a number of fundamental components and perform operations that fall within a limited number of categories. To facilitate the comparison of parallelization strategies for various meta-heuristic classes, it is convenient to define these common elements (Crainic and Toulouse, 2003):

- Initialization
- Neighbourhoods
- A neighbourhood selection criterion
- Candidate selection
- Acceptance criterion
- Stopping criteria

Similar to meta-heuristics generic the ACO approach attempts to solve an optimization problem by iterating the following two steps (Blum, 2005):

- candidate solutions are constructed using a pheromone model, that is, a parameterized probability distribution over the solution space;
- the candidate solutions are used to modify the pheromone values in a way that is deemed to bias future sampling toward high quality solutions.

According to Blum (2005), Ant Colony meta-heuristic algorithm pseudo-code is introduced as follows: 

**Algorithm 1.** Ant Colony Optimization (ACO)

```python
while termination_conditions_not_set:
    AntBasedSolutionConstruction()
    PheromoneUpdate()
    DeamonActions() #Optional
```

**Algorithm 2.** Ant Based Solution Construction

```python
s = []
Determine(N)
while N is not empty:
    c = ChooseFrom(N)
    extend s by appending solution component c
    Determine(N)
```

## 3. Traveling Salesperson Problem

The Traveling Salesperson Problem (TSP) is one of the most widely studied combinatorial optimization problems. Its statement is deceptively simple, and yet it remains one of the most challenging problems in OR (Laporte, 1992). An ACO application for TSP is introduced by Dorigo and Di Caro (1999) It is mentioned by them, since it is the first problem to be attacked by ACO methods, TSP plays a central role in ACO.

It can be found a definition of TSP in Laporte (1992): Let *G = (V, A)* be a graph where *V* is a set of *n* vertices. *A* is a set of arcs or edges, and let *C = (Cij)* be a distance (or cost) matrix associated with *A*. The TSP consists of determining a minimum distance circuit passing through each vertex once and only once. According to Blum (2005), the nodes *V* of this graph represent the cities, and the edge weights represent the distances between the cities. The goal is to find a closed path, called a tour, in G that contains each node exactly once and whose length is minimal. Thus, the search space consists of all tours in *G*. The objective function value of a tour is defined as the sum of the edge-weights of the edges that are in that tour. The TSP can be modelled in many different ways as a discrete optimization problem. The most common model consists of a binary decision variable *Xe* for each edge in *G.* If in a solution *Xe = 1*, then edge *e* is part of the tour that is defined by the solution.

## 4. Implementation

A generic symmetric TSP model is coded with "Python" language with "Numpy" library according to the algorithm mentioned above. Numpy is one of the suitable libraries for probabilistic selection. It is determined an iteration number to be used for termination condition of algorithm. The source code is available at https://github.com/ibrahimerdem/ModernHeuristicsFinal/blob/master/AntSystem.py (For repository: https://github.com/ibrahimerdem/ModernHeuristicsFinal)

In [1]:
def VeryBadTour(costs, number_of_cities):
    t = list(range(number_of_cities))
    return SolutionValue(costs, t)

According to Luke (2013), for the TSP, the ACO folks often set the initial value of pheromone is "popsize\*(1/Cost(D))", where popsize is number of cities constructing a trail, D is some costly, absurd tour like the Nearest Neighbor Tour. *VeryBadTour(costs, number_of_cities)* is used to calculate an absurd D value with using just an ordered sequence, t, where "costs" represents input data and "number_of_cities" respresents number of input cities.

In [2]:
def PheromoneUpdate(v, s):
    global pheromones
    global evaporation_constant
    for ix, i in np.ndenumerate(pheromones):
        pheromones[ix[0]][ix[1]] = (1-evaporation_constant) * i
    for j in range(len(s)-1):
        pheromones[s[j]][s[j+1]] += (1 / v)
        pheromones[s[j+1]][s[j]] += (1 / v)

Pheromone evaporation and pheromone deposit operations are performed by *PheromoneUpdate(v, s)*. It takes the solution value and solution sequence as input. The procedure also uses the global variables of pheromones matrix and "evaporation_constant". For symmetric TSPs, the distances between the cities *i*, and *j* are independent of the direction of traversing the edges (Dorigo and Stutzle, 2003). Since the distances between *i* to *j* and between *j* to *i* are the same, this edge is represented twice in the input data and the pheromones matrix, thus, the pheromone is deposited twice.

In [3]:
def SolutionValue(costs, s):
    v = 0
    for i in range(len(s)):
        if i == len(s)-1:
            v = v + costs[s[i]][s[0]]
        else:
            v = v + costs[s[i]][s[i+1]]
    return v

*SolutionValue(costs, s)* is the procedure that calculate the current solution's function value. It takes costs data and solution sequence as input. It returns the value, "v".  

In [4]:
def SolutionConstruction(l = None):
    if type(l) == int:
        s = [l]
    else:
        s = []
    N = Determine(s)
    while len(N) > 0:
        c = ChooseFrom(N, s)
        s.append(c)
        N = Determine(s)
    return s

def Determine(s):
    global number_of_cities
    N = list(range(number_of_cities))
    if len(s) > 0:
        for i in s:
            N.remove(i)
    return N

def ChooseFrom(N, s):
    global pheromones
    global alpha
    global beta
    probabilities = []
    total_factor = 0
    if len(s) > 0:
        current_loc = s[-1]    
        for n in N:
            phe = pheromones[current_loc][n]
            dis = costs[current_loc][n]
            total_factor += (np.power(phe, alpha) * np.power(1/dis, beta))
        for n in N:
            phe = pheromones[current_loc][n]
            dis = costs[current_loc][n]
            decision_probability = (np.power(phe, alpha) * np.power(1/dis, beta)) / total_factor #pheromone model
            probabilities.append(decision_probability)
        c = np.random.choice(N, p=probabilities, size=1)[0] #probabilistic decision for next state
    else:
        c = np.random.choice(N) #random starting location if it is not set
    return c

*AntSystem()* uses *SolutionConstruction(l = starting_location)* procedure to construct a candidate solution. The procedure determines the list of available states with the procedure *Determine(s)*, and choses the next state from the list of available states with the procedure of *ChooseFrom(). SolutionConstruction()* can take an optional argument of starting location if it is preferred the determined starting position information as an input. *SolutionConstruction()* returns with the new candidate solution sequence of “s”. "total_factor" respresents the denominator of the composition of the pheromone trails and heuristic values seen below (Dorigo and Di Caro, 1999):

![pic1.png](pictures/pic1.png)

The heuristic values are the distance between cities i and j seen below. In other words, the shorter the distance between two cities i and j, the higher the heuristic value (Dorigo and Di Caro, 1999).

![pic2.png](pictures/pic2.png)

To construct candidate solutions *ChooseFrom(N, s)* procedure takes two argument, “N” is a list of possible cities in order to produce a feasible solution, and “s” is represents current sequence so far. Before choosing, it must be run *Determine()* procedure which takes existing sequence of trail so far as an argument.    

In [5]:
import numpy as np

evaporation_constant = 0.5
number_of_cities = 2
pheromones = np.ones((number_of_cities, number_of_cities))
alpha = 1
beta = 2

def Initialization(costs, e, a, b):
    global evaporation_constant
    evaporation_constant = e
    global number_of_cities
    number_of_cities = len(costs)
    global alpha
    alpha = a
    global beta
    beta = b
    global pheromones
    D = VeryBadTour(costs, number_of_cities)
    initial_pheromone_value = number_of_cities / D
    initial_pheromones = np.ones((number_of_cities, number_of_cities))
    initial_pheromones.fill(initial_pheromone_value)
    pheromones = initial_pheromones

def AntSystem(costs, e = 0.5, n = 10, alpha = 1, beta = 5, starting_location = None):
    Initialization(costs, e, alpha, beta)
    best_solution = []
    best_value = VeryBadTour(costs, number_of_cities)
    while n > 0:
        s = SolutionConstruction(l = starting_location)
        v = SolutionValue(costs, s)
        if v < best_value or len(best_solution) == 0:
            s.append(s[0])
            best_solution = s
            best_value = v
        PheromoneUpdate(v, s)
        n = n - 1
    print(f'{best_solution} is the best tour and {best_value} is the best value')

The main block of code is *AntSystem(costs, e = 0.5, n = 100, alpha = 1, beta = 1, starting_location = None)*, where inputs: 

- "costs" is a matrix (n, n) where n is number of cities, representing all combination of distances between cities,
- "e" is evaporation rate with default value of .5, 
- "n" is number of iteration of algorithm with default value of 100,
- "alpha and beta" are the parameters to control the relative weight of pheromone trail and heuristic value (Dorigo and Di Caro, 1999) and their default values are 1 and 5 respectively (Dorigo, 1992), 
- "starting_location" can be take an integer value which can be used in order to start the tour with specific location, but the default value of it is "None". 

Procedure evaluates different solution values to find the least and return a sequence “best_solution” representing the cities where the salesperson going through and finished a tour, and a value of objective function “best_value”.

*Initialization(costs, e, alpha, beta)* sets the initial values of global variables and "pheromones" matrix.

## 5. Tests and Results

For testing the implementation it is chosen a problem example from Winston's Operations Research book (2004). One can be found in the book a small sized TSP, and a linear programming construction of it.  

**Example.** Joe State lives in Gary, Indiana. He owns insurance agencies in Gary, Fort Wayne, Evansville, Terre Haute, and South Bend. Each December, he visits each of his insurance agencies. The distance between each agency (in miles) is shown in table below. What order of visiting his agencies will minimize the total distance traveled ?

| | Gary | Fort Wayne | Evansville | Terre Haute | South Bend |
|---|---|---|---|---|---|
|**Gary** | 0 | 132 | 217 | 164 | 58 |
|**Fort Wayne** | 132 | 0 | 290 | 201 | 79 |
|**Evansville** | 217 | 290 | 0 | 113 | 303 |
|**Terre Haute** | 164 | 201 | 113 | 0 | 196 |
|**South Bend** | 58 | 79 | 303 | 196 | 0 |

For solution, let numbers \[0, 4\] are represent the cities respectively in order to construct cost matrix. According to Winstons's example the starting location salesperson corresponding to number 0, thus, the starting_location is set to be 0. With the inputs of cost matrix seen below, 100 of iteration, alpha=1, beta=5, and 0.5 of evaporation constant, the outputs of all trials is 0-4-1-3-2-0 sequence with 668 miles cost value. According to Winston, it is obtained that algorithm results with an optimum solution.

In [6]:
costs = [[0, 132, 217, 164, 58],
        [132, 0, 290, 201, 79],
        [217, 290, 0, 113, 303],
        [164, 201, 113, 0, 196],
        [58, 79, 303, 196, 0]]

for i in range(10):
    AntSystem(costs, n=100, starting_location=0) #Starts at 0

[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value


With the same inputs, without starting_location, which means with random starting location, it is obtained twice 1-4-0-3-2-1 route with 704 miles, once 3-2-1-4-0-3 route with 704 miles, twice 3-2-0-4-1-3 route with 668 miles, once 2-3-1-4-0-2 route with 668 miles, once 3-1-4-0-2-3 route with 668 miles, and 3 times 0-4-1-3-2-0 with 668 miles outputs of 10 trials.

In [7]:
costs = [[0, 132, 217, 164, 58],
        [132, 0, 290, 201, 79],
        [217, 290, 0, 113, 303],
        [164, 201, 113, 0, 196],
        [58, 79, 303, 196, 0]]

for i in range(10):
    AntSystem(costs, n=100) #Starts randomly

[3, 2, 0, 4, 1, 3] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[3, 2, 0, 4, 1, 3] is the best tour and 668 is the best value
[1, 4, 0, 3, 2, 1] is the best tour and 704 is the best value
[1, 4, 0, 3, 2, 1] is the best tour and 704 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[3, 1, 4, 0, 2, 3] is the best tour and 668 is the best value
[0, 4, 1, 3, 2, 0] is the best tour and 668 is the best value
[3, 2, 1, 4, 0, 3] is the best tour and 704 is the best value


For further analysis the with same data, it is tested a drawback of nearest-neighbor heuristic (NNH) mentioned by Winston. With NNH, and starting_location value 2 it is obtained 2-3-0-4-1-2 route with 704 miles corresponding to a nonoptimal solution. However, with ACO, it is obtained 5 times 2-3-1-4-0-2 route with 668 miles, and 5 times 2-3-0-4-1-2 route with 704 miles outputs of 10 trials. In addition, with 1000 of iterations, it is obtained 5 times 2-3-1-4-0-2 route with 668 miles, 4 times 2-3-0-4-1-2 route with 704 miles output of 10 trials.

In [8]:
costs = [[0, 132, 217, 164, 58],
        [132, 0, 290, 201, 79],
        [217, 290, 0, 113, 303],
        [164, 201, 113, 0, 196],
        [58, 79, 303, 196, 0]]

for i in range(10):
    AntSystem(costs, n=1000, starting_location=2) #Starts at 2, and 1000 iterations

[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 0, 4, 1, 2] is the best tour and 704 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 0, 4, 1, 2] is the best tour and 704 is the best value
[2, 3, 1, 4, 0, 2] is the best tour and 668 is the best value
[2, 3, 0, 4, 1, 2] is the best tour and 704 is the best value


Another example is gathered from Google OR-Tools and tested with the data of 12 locations (Available at: https://developers.google.com/optimization/routing/tsp). Cities are represented by integers as: 0 New York, 1 Los Angeles, 2 Chicago, 3 Minneapolis, 4 Denver, 5 Dallas, 6 Seattle, 7 Boston, 8 San Francisco, 9 St. Louis, 10 Houston, 11 Phoenix, 12 Salt Lake City. As a results of 20 experiments with starting point of 0, 0-7-2-9-5-10-11-1-8-6-12-4-3-0 route with the value 7343 miles is obtained. However, optimal solution value can be seen as 7293 miles.

In [9]:
costs = [[0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0]]

for i in range(20):
    AntSystem(costs, n=100, starting_location=0)

[0, 7, 2, 9, 10, 5, 4, 12, 11, 1, 8, 6, 3, 0] is the best tour and 7619 is the best value
[0, 7, 2, 3, 4, 12, 11, 1, 8, 6, 10, 5, 9, 0] is the best tour and 7970 is the best value
[0, 7, 2, 9, 3, 4, 12, 11, 1, 8, 6, 5, 10, 0] is the best tour and 8131 is the best value
[0, 7, 2, 9, 3, 4, 12, 6, 8, 1, 11, 5, 10, 0] is the best tour and 7534 is the best value
[0, 7, 2, 9, 5, 10, 11, 1, 8, 6, 12, 4, 3, 0] is the best tour and 7343 is the best value
[0, 7, 2, 3, 4, 12, 11, 1, 8, 6, 10, 5, 9, 0] is the best tour and 7970 is the best value
[0, 7, 2, 9, 10, 5, 4, 12, 8, 1, 11, 6, 3, 0] is the best tour and 8151 is the best value
[0, 7, 2, 9, 5, 10, 11, 1, 8, 6, 12, 4, 3, 0] is the best tour and 7343 is the best value
[0, 7, 2, 9, 10, 5, 11, 1, 8, 6, 12, 4, 3, 0] is the best tour and 7345 is the best value
[0, 7, 2, 9, 10, 5, 4, 12, 11, 1, 8, 6, 3, 0] is the best tour and 7619 is the best value
[0, 7, 2, 9, 5, 10, 4, 12, 11, 1, 8, 6, 3, 0] is the best tour and 7703 is the best value
[0, 7, 2, 

Different size of data can be gathered from TSPLIB. "Bays29" represents the data of 29 cities in Bavaria, street distances (Groetschel,Juenger,Reinelt)(Available at:http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/bays29.tsp). According to 20 experiments with this data and starting location of 0, 0-27-5-11-8-4-25-28-2-1-19-9-3-14-17-13-21-16-10-18-15-24-6-22-26-23-12-20-7-0 sequence is obtained once with the value of 2288 miles. On the other hand the optimal solution value of Bays29 reported as 2020 miles.(Optimal solutions are available at: http://elib.zib.de/pub/mp-testdata/tsp/tsplib/stsp-sol.html)

In [10]:
costs = [[0, 107, 241, 190, 124, 80, 316, 76, 152, 157, 283, 133, 113, 297, 228, 129, 348, 276, 188, 150, 65, 341, 184, 67, 221, 169, 108, 45, 167],
        [107, 0, 148, 137, 88, 127, 336, 183, 134, 95, 254, 180, 101, 234, 175, 176, 265, 199, 182, 67, 42, 278, 271, 146, 251, 105, 191, 139, 79],
        [241, 148, 0, 374, 171, 259, 509, 317, 217, 232, 491, 312, 280, 391, 412, 349, 422, 356, 355, 204, 182, 435, 417, 292, 424, 116, 337, 273, 77],
        [190, 137, 374, 0, 202, 234, 222, 192, 248, 42, 117, 287, 79, 107, 38, 121, 152, 86, 68, 70, 137, 151, 239, 135, 137, 242, 165, 228, 205],
        [124, 88, 171, 202, 0, 61, 392, 202, 46, 160, 319, 112, 163, 322, 240, 232, 314, 287, 238, 155, 65, 366, 300, 175, 307, 57, 220, 121, 97],
        [80, 127, 259, 234, 61, 0, 386, 141, 72, 167, 351, 55, 157, 331, 272, 226, 362, 296, 232, 164, 85, 375, 249, 147, 301, 118, 188, 60, 185],
        [316, 336, 509, 222, 392, 386, 0, 233, 438, 254, 202, 439, 235, 254, 210, 187, 313, 266, 154, 282, 321, 298, 168, 249, 95, 437, 190, 314, 435],
        [76, 183, 317, 192, 202, 141, 233, 0, 213, 188, 272, 193, 131, 302, 233, 98, 344, 289, 177, 216, 141, 346, 108, 57, 190, 245, 43, 81, 243],
        [152, 134, 217, 248, 46, 72, 438, 213, 0, 206, 365, 89, 209, 368, 286, 278, 360, 333, 284, 201, 111, 412, 321, 221, 353, 72, 266, 132, 111],
        [157, 95, 232, 42, 160, 167, 254, 188, 206, 0, 159, 220, 57, 149, 80, 132, 193, 127, 100, 28, 95, 193, 241, 131, 169, 200, 161, 189, 163],
        [283, 254, 491, 117, 319, 351, 202, 272, 365, 159, 0, 404, 176, 106, 79, 161, 165, 141, 95, 187, 254, 103, 279, 215, 117, 359, 216, 308, 322],
        [133, 180, 312, 287, 112, 55, 439, 193, 89, 220, 404, 0, 210, 384, 325, 279, 415, 349, 285, 217, 138, 428, 310, 200, 354, 169, 241, 112, 238],
        [113, 101, 280, 79, 163, 157, 235, 131, 209, 57, 176, 210, 0, 186, 117, 75, 231, 165, 81, 85, 92, 230, 184, 74, 150, 208, 104, 158, 206],
        [297, 234, 391, 107, 322, 331, 254, 302, 368, 149, 106, 384, 186, 0, 69, 191, 59, 35, 125, 167, 255, 44, 309, 245, 169, 327, 246, 335, 288],
        [228, 175, 412, 38, 240, 272, 210, 233, 286, 80, 79, 325, 117, 69, 0, 122, 122, 56, 56, 108, 175, 113, 240, 176, 125, 280, 177, 266, 243],
        [129, 176, 349, 121, 232, 226, 187, 98, 278, 132, 161, 279, 75, 191, 122, 0, 244, 178, 66, 160, 161, 235, 118, 62, 92, 277, 55, 155, 275],
        [348, 265, 422, 152, 314, 362, 313, 344, 360, 193, 165, 415, 231, 59, 122, 244, 0, 66, 178, 198, 286, 77, 362, 287, 228, 358, 299, 380, 319],
        [276, 199, 356, 86, 287, 296, 266, 289, 333, 127, 141, 349, 165, 35, 56, 178, 66, 0, 112, 132, 220, 79, 296, 232, 181, 292, 233, 314, 253],
        [188, 182, 355, 68, 238, 232, 154, 177, 284, 100, 95, 285, 81, 125, 56, 66, 178, 112, 0, 128, 167, 169, 179, 120, 69, 283, 121, 213, 281],
        [150, 67, 204, 70, 155, 164, 282, 216, 201, 28, 187, 217, 85, 167, 108, 160, 198, 132, 128, 0, 88, 211, 269, 159, 197, 172, 189, 182, 135],
        [65, 42, 182, 137, 65, 85, 321, 141, 111, 95, 254, 138, 92, 255, 175, 161, 286, 220, 167, 88, 0, 299, 229, 104, 236, 110, 149, 97, 108],
        [341, 278, 435, 151, 366, 375, 298, 346, 412, 193, 103, 428, 230, 44, 113, 235, 77, 79, 169, 211, 299, 0, 353, 289, 213, 371, 290, 379, 332],
        [184, 271, 417, 239, 300, 249, 168, 108, 321, 241, 279, 310, 184, 309, 240, 118, 362, 296, 179, 269, 229, 353, 0, 121, 162, 345, 80, 189, 342],
        [67, 146, 292, 135, 175, 147, 249, 57, 221, 131, 215, 200, 74, 245, 176, 62, 287, 232, 120, 159, 104, 289, 121, 0, 154, 220, 41, 93, 218],
        [221, 251, 424, 137, 307, 301, 95, 190, 353, 169, 117, 354, 150, 169, 125, 92, 228, 181, 69, 197, 236, 213, 162, 154, 0, 352, 147, 247, 350],
        [169, 105, 116, 242, 57, 118, 437, 245, 72, 200, 359, 169, 208, 327, 280, 277, 358, 292, 283, 172, 110, 371, 345, 220, 352, 0, 265, 178, 39],
        [108, 191, 337, 165, 220, 188, 190, 43, 266, 161, 216, 241, 104, 246, 177, 55, 299, 233, 121, 189, 149, 290, 80, 41, 147, 265, 0, 124, 263],
        [45, 139, 273, 228, 121, 60, 314, 81, 132, 189, 308, 112, 158, 335, 266, 155, 380, 314, 213, 182, 97, 379, 189, 93, 247, 178, 124, 0, 199],
        [167, 79, 77, 205, 97, 185, 435, 243, 111, 163, 322, 238, 206, 288, 243, 275, 319, 253, 281, 135, 108, 332, 342, 218, 350, 39, 263, 199, 0]]

for i in range(20):
    AntSystem(costs, n=100, starting_location=0)

[0, 27, 5, 11, 8, 4, 25, 28, 1, 20, 19, 9, 3, 14, 17, 13, 21, 16, 10, 18, 15, 23, 26, 7, 22, 6, 24, 12, 2, 0] is the best tour and 2434 is the best value
[0, 27, 5, 11, 8, 4, 25, 28, 2, 1, 20, 12, 9, 19, 3, 14, 18, 15, 26, 7, 23, 22, 6, 24, 10, 21, 13, 17, 16, 0] is the best tour and 2317 is the best value
[0, 27, 5, 11, 8, 4, 25, 28, 2, 1, 20, 12, 9, 19, 3, 14, 17, 13, 21, 16, 10, 18, 15, 26, 7, 23, 22, 6, 24, 0] is the best tour and 2241 is the best value
[0, 27, 5, 11, 8, 4, 25, 28, 1, 20, 12, 9, 3, 14, 18, 15, 26, 7, 23, 22, 6, 24, 10, 21, 13, 17, 16, 19, 2, 0] is the best tour and 2410 is the best value
[0, 27, 4, 8, 11, 5, 25, 28, 2, 20, 1, 19, 9, 3, 14, 13, 17, 16, 21, 10, 18, 15, 26, 23, 7, 22, 6, 24, 12, 0] is the best tour and 2287 is the best value
[0, 27, 5, 8, 4, 25, 28, 2, 1, 20, 19, 9, 3, 14, 17, 13, 21, 16, 10, 18, 24, 6, 22, 7, 26, 23, 15, 12, 11, 0] is the best tour and 2258 is the best value
[0, 27, 5, 11, 4, 8, 25, 28, 2, 1, 20, 19, 9, 3, 14, 18, 24, 15, 26, 23, 12,

## 6. Conclusion

Experiments show that ACO with the small size of data as in the example, capable of constructing optimal solution with a non-trivial probability. Furthermore, with relatively high number of iterations, one can notice that the probability of optimal solution can be increased. On the other hand with a bit more sizeble data results can only approach the optimum solution. Since the obtaining probibility of optimal solution decreases, the capability of avoiding local extremum obstacles of ACO decreases with given inputs. However, it should be tested same data with different values of parameters alpha and beta.

As it is mentioned by Blum (2005), even though the original AS algorithm achieved encouraging results for the TSP problem, it is found to be inferior to state-of-the-art algorithms for the TSP as well as for other CO problems. Therefore, several extensions and improvements of the original AS algorithm were introduced over the years. As an extension a "hill-climbing" effort can be assembled to the original algorithm as proposed by Luke (2013).

As a fairly young research field, it is expected a meta-heuristic some desirable properties. In terms of desirable properties (Hansen and Mladenovic, 2003), ACO meets most of desires with its simplicity, precision, coherence and effectiveness, but with this initial version, AS, it can be considered as efficiency and robustness are not completed. However, one can be overcome these drawbacks with extensions and improved versions, in terms of innovation.    

## References

1. Marco D. (1992) Optimization, Learning and Natural Algorithms, PhD thesis, Politecnico di Milano, Italy
2. Laporte G. (1992), The traveling salesman problem:  An overview of exact and approximate algorithms, European Journal of Operational Research, 59(2):231–247.
3. Dorigo M. and Di Caro G. (1999), Ant colony optimization:  A new meta-heuristic. page 1477 Vol. 2
4. Dorigo M. and Stutzle T. (2003), The ant colony optimization metaheuristic:  algo-rithms, applications, and advances. Handbook of Metaheuristics.
5. Hansen P. and Mladenovic N. (2003), Variable neighborhood search, Handbook of Metaheuristics.
6. Crainic T. G. and Toulouse M. (2003), Parallel strategies for meta-heuristics, Hand-book of Metaheuristics.
7. Winston W. L. (2004), Operations Research Applications and Algorithms (fourth edition), Thomson
8. Christian B. (2005), Ant colony optimization:  Introduction and recent trends, Physics of Life Reviews, 2(4):353–373.
9. Luke S. (2013), Essentials of Metaheuristics (second edition). Lulu, Available for free at http://cs.gmu.edu/∼sean/book/metaheuristics/